In [43]:
import pandas as pd
import numpy as np

In [44]:
MERIDIAN_PATH = "../../data/life/Bite Off More Than You Can Chew Braces profile - Sheet1.csv"
TEETH_UP_PATH = "../../data/life/BOMTYCC_teeth_up_1854211842023.json"
TEETH_DOWN_PATH = "../../data/life/BOMTYCC_teeth_down_1826391842023.json"

In [49]:
teeth_up = pd.read_json(TEETH_UP_PATH)
teeth_down = pd.read_json(TEETH_DOWN_PATH)
teeth_df = pd.concat([teeth_up, teeth_down])
teeth_df = teeth_df.reset_index(drop=True)
teeth_data = teeth_df["data"]

,canvasWidth,canvasHeight,teethDirection,data
0,2000,2000,up,"{'toothId': 18, 'toothPositions': [[238, 1407]..."
1,2000,2000,up,"{'toothId': 17, 'toothPositions': [[238, 1407]..."
2,2000,2000,up,"{'toothId': 16, 'toothPositions': [[238, 1407]..."
3,2000,2000,up,"{'toothId': 15, 'toothPositions': [[238, 1407]..."
4,2000,2000,up,"{'toothId': 14, 'toothPositions': [[238, 1407]..."
5,2000,2000,up,"{'toothId': 13, 'toothPositions': [[238, 1407]..."
6,2000,2000,up,"{'toothId': 12, 'toothPositions': []}"
7,2000,2000,up,"{'toothId': 11, 'toothPositions': []}"
8,2000,2000,up,"{'toothId': 21, 'toothPositions': [[238, 1407]..."
9,2000,2000,up,"{'toothId': 22, 'toothPositions': [[238, 1407]..."


In [50]:
teeth_df["data"][0]

{'toothId': 18, 'toothPositions': [[238, 1407], [324, 1153], [388, 884]]}

In [14]:
meridian_df = pd.read_csv(CSV_PATH)

In [15]:
# remove nulls
meridian_df.dropna(subset=["Tooth number", "Domain"], inplace=True)

In [16]:
# teeth indices to ints
f = lambda s: [int(s) for s in s.split("-")]
meridian_df["Tooth number"] = meridian_df["Tooth number"].apply(f)

In [21]:
teeth_df["data"][0]

{'toothId': 48, 'toothPositions': [[238, 1407], [324, 1153], [388, 884]]}

In [42]:
# get teeth ids associated with Domain 5
ids = meridian_df["Tooth number"][5]
# get domain name
domain = meridian_df["Domain"][5]

mask = teeth_df["data"].apply(lambda x: x['toothId'] in ids)
teeth = teeth_df[mask]
teeth

#  look at all the teeth 
#  each tooth has tooth and and teeth positions

# CREATE the TRAIN DATASET
# take a domain (from meridian df)
# take the IDs of associated teeth from this df
# look for position of each associated teeth

#for index, row in teeth_df["data"].iteritems():
#   toothId = int(row['toothId'])
#    print(row)
    # match each tooth with 
    #if toothId in excitement_teeth_ids:
    #  print(toothId)
#excitement_teeth_ids    

,canvasWidth,canvasHeight,teethDirection,data
0,2000,2000,low,"{'toothId': 48, 'toothPositions': [[238, 1407]..."
15,2000,2000,low,"{'toothId': 38, 'toothPositions': [[882, 938],..."


In [36]:
# filtered_series = teeth_df["data"][teeth_df["data"].toothId == 48]
# filtered_s = s[s.apply(lambda x: x['Age'] > 30)]



#f = lambda s: [v["toothId"] == 48 for v in s]
mask = teeth_df["data"].apply(lambda x: x['toothId'] == 48)
d = teeth_df[mask]
d

,canvasWidth,canvasHeight,teethDirection,data


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(meridian_df["Tooth number"])